<a href="https://colab.research.google.com/github/Priyo-prog/Complete-Feature-Engineering/blob/master/Missing%20Data%20Imputation/imputation_with_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatic Selection of best Imputation technique with ScikitLearn**

In this lesson we will use grid search over imputation methods available in Scikit-Learn to determine which imputation technique works best for this dataset and the machine learning model of choice.

We will train a very simple ML model as part of a small pipeline.

## **Import the libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## **Import the dataset**

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# get the filename
filename = "/content/drive/MyDrive/Data Science/Feature Selection/houseprice.csv"

In [4]:
data = pd.read_csv(filename)
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# find the categorical features
# those columns with datatype "Object"
categorical_features = [cat for cat in data.columns if data[cat].dtype == "O"]

# find the numerical variables
# those columns with datatype not "Object" and also exclude target "SalePrice"
numerical_features = [num for num in data.columns if data[num].dtype != "O" and num != "SalePrice"]

In [6]:
data[categorical_features].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [7]:
data[numerical_features].head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


In [23]:
numerical_features

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [8]:
# Separate in train and test split
X_train, X_test, y_train, y_test = train_test_split(
                                   data.drop(labels=["SalePrice"], axis=1),
                                   data["SalePrice"],
                                   test_size=0.3,
                                   random_state=0)

In [9]:
X_train.shape, X_test.shape

((1022, 80), (438, 80))

In [10]:
from sklearn.pipeline import Pipeline

## **Pipelines and ColumnTransformer for Preprocessing**

In [11]:
# We create pipelines for both numerical and categorical features

numerical_transform = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())])

categorical_transform = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(
                 transformers=[
                    ("numerical", numerical_transform, numerical_features),
                    ("categorical", categorical_transform, categorical_features)])

## **Pipeline for Preprocessor and Classifier**

In [13]:
# Append classifier to preprocessing pipeline

clf = Pipeline(
       steps = [
          ("preprocess", preprocessor),
          ("classifier", Lasso(max_iter=2000))])

## **Create GridSearch for better Imputation method**

In [18]:
# Create the grid search dictionary
param_grid = {
    "preprocessor__numerical__imputer__strategy": ["mean", "median"],
    "preprocessor__categorical__imputer__strategy": ["most_frequent", "constant"],
    "classifier__alpha": [10,100,200]
}

In [19]:
grid_search = GridSearchCV(clf, param_grid, cv=5,  n_jobs=-1, scoring="r2")

In [20]:
# Now fit the data and find the best parameter
grid_search.fit(X_train, y_train)

# print the best score over the train set
print(("best liner regression from grid search: %.3f"
       % grid_search.score(X_train, y_train)))

ValueError: ignored